<a href="https://colab.research.google.com/github/srijit43/NLP-theory-and-code/blob/main/05_07Comparing_KFold_and_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Adjustment before proceeding

In [3]:
X_tfidf_feat = X_tfidf_feat.rename(str,axis='columns')
X_count_feat = X_count_feat.rename(str,axis='columns')

### Exploring parameter settings using GridSearchCV

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
rf = RandomForestClassifier()
params = {'n_estimators':[10,100,200,300],
          'max_depth':[10,20,30,60,None]}

gs = GridSearchCV(rf,params,cv=5,n_jobs=-1)

gs_fit = gs.fit(X_tfidf_feat,data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
18,27.114998,0.682605,0.398114,0.084369,None,200,"{'max_depth': None, 'n_estimators': 200}",0.976661,0.978456,0.974843,0.968553,0.972147,0.974132,0.003483,1
17,13.667249,0.237289,0.272724,0.056590,None,100,"{'max_depth': None, 'n_estimators': 100}",0.978456,0.977558,0.973944,0.968553,0.966757,0.973054,0.004694,2
14,19.885796,0.778202,0.357436,0.091976,60,200,"{'max_depth': 60, 'n_estimators': 200}",0.976661,0.975763,0.970350,0.970350,0.971249,0.972875,0.002759,3
13,10.125926,0.749133,0.250100,0.034460,60,100,"{'max_depth': 60, 'n_estimators': 100}",0.977558,0.975763,0.972147,0.967655,0.969452,0.972515,0.003718,4
12,1.597424,0.377890,0.207981,0.075114,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.973070,0.975763,0.974843,0.964960,0.973046,0.972336,0.003834,5


In [6]:
rf = RandomForestClassifier()
params = {'n_estimators':[10,100,200,300],
          'max_depth':[10,20,30,60,None]}

gs = GridSearchCV(rf,params,cv=5,n_jobs=-1)

gs_fit = gs.fit(X_count_feat,data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
18,26.907992,0.688451,0.329471,0.006156,None,200,"{'max_depth': None, 'n_estimators': 200}",0.981149,0.973968,0.973046,0.969452,0.970350,0.973593,0.004128,1
19,39.265781,2.267107,0.509768,0.171461,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.974865,0.973944,0.967655,0.969452,0.972695,0.003629,2
17,14.095634,1.038623,0.276981,0.064851,None,100,"{'max_depth': None, 'n_estimators': 100}",0.978456,0.976661,0.974843,0.966757,0.966757,0.972694,0.004981,3
14,20.119127,1.135417,0.341713,0.103273,60,200,"{'max_depth': 60, 'n_estimators': 200}",0.974865,0.974865,0.972147,0.967655,0.969452,0.971797,0.002885,4
15,29.033007,0.269019,0.460533,0.151924,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.974865,0.973070,0.972147,0.966757,0.966757,0.970719,0.003351,5
